In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip -P ../data/raw/

'wget' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [2]:
!mkdir ../data/processed
!python3 ../src/data/preprocess_data.py -i  ../data/raw/OpportunityUCIDataset.zip -o ../processed/oppChallenge_gestures.data

SyntaxError: invalid syntax (<ipython-input-2-05c2b0460174>, line 1)

### Setup

In [10]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import _pickle as cp
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.metrics as metrics

import torch
from torch import nn
import torch.nn.functional as F

#NB_SENSOR_CHANNELS = 113
NB_SENSOR_CHANNELS = 15
SLIDING_WINDOW_LENGTH = 50
SLIDING_WINDOW_STEP = 25

In [11]:
import os
import sys

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

from data.sliding_window import sliding_window

## Load in Data

In [12]:

def load_dataset(filename):
    
    with open(filename, 'rb') as f:
        data = cp.load(f)
    
    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)
    
    print("..part of x_train",type(X_train))
    #print(X_train[:23,:2])
    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

print("Loading data...")
#X_train, y_train, X_test, y_test = load_dataset('../data/processed/oppChallenge_gestures.data')
X_train, y_train, X_test, y_test = load_dataset('C:\\Users\\Zhiha\\Documents\\file_only\\research\\tool_program\\ori\\Sensor-Based-Human-Activity-Recognition-DeepConvLSTM-Pytorch\\data\\processed\\mygestures.data')


Loading data...
 ..from file C:\Users\Zhiha\Documents\file_only\research\tool_program\ori\Sensor-Based-Human-Activity-Recognition-DeepConvLSTM-Pytorch\data\processed\mygestures.data
 ..reading instances: train (101668, 15), test (43573, 15)
..part of x_train <class 'numpy.ndarray'>


## Data Segmentation

In [13]:
assert NB_SENSOR_CHANNELS == X_train.shape[1]
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x, (ws, data_x.shape[1]), (ss, 1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y, ws, ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# Sensor data is segmented using a sliding window mechanism
X_train, y_train = opp_sliding_window(X_train, y_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
X_test, y_test = opp_sliding_window(X_test, y_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
#print(X_train[:24,:3])
print(X_train.shape)
# Data is reshaped
X_train = X_train.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)) # for input to Conv1D
X_test = X_test.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)) # for input to Conv1D

print(y_train[2000:2500,])
print(" ..after sliding and reshaping, train data: inputs {0}, targets {1}".format(X_train.shape, y_train.shape))
print(" ..after sliding and reshaping, test data : inputs {0}, targets {1}".format(X_test.shape, y_test.shape))

(4065, 50, 15)
[ 2  2  2  2  9  9  9 13 13 13 14 14 14 14 14 14  4  4  4  7  7  7  7  7
  7  7  7  7  2  2  2  2  2  2  2  2  0  0  0  3  3  3  3  1  1  1 14 14
 14 14 14 14 14 14  3  3  3  3  4  4  4  4  8  8  8  8 12 12 12 12 12 12
 12 12 12  1  1  1  2  2  2  2  2  2  8  8  8  8 11 11 11  5  5  5  0  0
  0  0 11 11 11  9  9  9  5  5  5  0  0  0  0  0  0  0  0  0 12 12 12 12
 12 12 12 12 12 11 11 11 11 11 11  9  9  9  4  4  4  4 11 11 11  3  3  3
  3 14 14 14 14 14  1  1  1  4  4  4  4  4  4  9  9 10 10 10 10 10 14 14
 14 14 14 14  1  1  1 14 14 14 14 14 14 14 14 14  7  7  7  7  7  7  7  7
  7  7  7  7  7  7  7 13 13 13 13 13 13 11 11 11  8  8  8  8 10 10 10  6
  6  6  6  4  4  4  5  5  1  1  1  1  9  9  9 11 11 11 11 11  9  9  9  9
  4  4  4 11 11 11 11  9  9  5  5  5  6  6  6 14 14 14 14 14 14  1  1 10
 10 10 10  5  5  3  3  3  3  9  9  9 12 12 12 12 12 12 12 12 12 12 12 12
 12 12 10 10 10  5  5  5  3  3  3  3 10 10 10 10  1  1  1 13 13 13 13  5
  5  5 14 14 14 14  0  0  0  4  4  4

## Define the Model

In [19]:
class HARModel(nn.Module):
    
    def __init__(self, n_hidden=128, n_layers=1, n_filters=64, 
                 n_classes=15, filter_size=5, drop_prob=0.5):
        super(HARModel, self).__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.n_filters = n_filters
        self.n_classes = n_classes
        self.filter_size = filter_size
             
        self.conv1 = nn.Conv1d(NB_SENSOR_CHANNELS, n_filters, filter_size)
        self.conv2 = nn.Conv1d(n_filters, n_filters, filter_size)
        self.conv3 = nn.Conv1d(n_filters, n_filters, filter_size)
        self.conv4 = nn.Conv1d(n_filters, n_filters, filter_size)
        
        self.lstm1  = nn.LSTM(n_filters, n_hidden, n_layers)
        self.lstm2  = nn.LSTM(n_hidden, n_hidden, n_layers)
        
        self.fc = nn.Linear(n_hidden, n_classes)

        self.dropout = nn.Dropout(drop_prob)
    
    def forward(self, x, hidden, batch_size):
        
        x = x.view(-1, NB_SENSOR_CHANNELS, SLIDING_WINDOW_LENGTH)#reshape
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        #print(x.size())
        x = x.view(34, -1, self.n_filters)
        x, hidden = self.lstm1(x, hidden)
        x, hidden = self.lstm2(x, hidden)
        
        x = x.contiguous().view(-1, self.n_hidden)
        x = self.dropout(x)
        x = self.fc(x)
        
        out = x.view(batch_size, -1, self.n_classes)[:,-1,:]
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        '''
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        '''
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
    
net = HARModel()

## Initialize Model Weights

In [20]:
def init_weights(m):
    if type(m) == nn.LSTM:
        for name, param in m.named_parameters():
            if 'weight_ih' in name:
                torch.nn.init.orthogonal_(param.data)
            elif 'weight_hh' in name:
                torch.nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                param.data.fill_(0)
    elif type(m) == nn.Conv1d or type(m) == nn.Linear:
        torch.nn.init.orthogonal_(m.weight)
        m.bias.data.fill_(0)
net.apply(init_weights)    

HARModel(
  (conv1): Conv1d(15, 64, kernel_size=(5,), stride=(1,))
  (conv2): Conv1d(64, 64, kernel_size=(5,), stride=(1,))
  (conv3): Conv1d(64, 64, kernel_size=(5,), stride=(1,))
  (conv4): Conv1d(64, 64, kernel_size=(5,), stride=(1,))
  (lstm1): LSTM(64, 128)
  (lstm2): LSTM(128, 128)
  (fc): Linear(in_features=128, out_features=15, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

## Generate Minibatches

In [21]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=True):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [22]:
## check if GPU is available
#train_on_gpu = torch.cuda.is_available()
#if(train_on_gpu):
    #print('Training on GPU!')
#else: 
    #print('No GPU available, training on CPU; consider making n_epochs very small.')

## Train the Network

In [23]:
def train(net, epochs=100, batch_size=10, lr=0.01):
    
    opt = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss()
    
    #if(train_on_gpu):
        #net.cuda()
     
    for e in range(epochs):
        
        # initialize hidden state
        h = net.init_hidden(batch_size)         
        train_losses = []    
        net.train()
        for batch in iterate_minibatches(X_train, y_train, batch_size):
            x, y = batch

            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            #if(train_on_gpu):
            #        inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])
            
            # zero accumulated gradients
            opt.zero_grad()   
            
            # get the output from the model
            output, h = net(inputs, h, batch_size)
            
            loss = criterion(output, targets.long())
            train_losses.append(loss.item())
            loss.backward()
            opt.step()
            
        val_h = net.init_hidden(batch_size)
        val_losses = []
        accuracy=0
        f1score=0
        net.eval()
        with torch.no_grad():
            for batch in iterate_minibatches(X_test, y_test, batch_size):
                x, y = batch     

                inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

                val_h = tuple([each.data for each in val_h])

                #if(train_on_gpu):
                #    inputs, targets = inputs.cuda(), targets.cuda()
                    
                output, val_h= net(inputs, val_h, batch_size)

                val_loss = criterion(output, targets.long())
                val_losses.append(val_loss.item())

                top_p, top_class = output.topk(1, dim=1)
                equals = top_class == targets.view(*top_class.shape).long()
                accuracy += torch.mean(equals.type(torch.FloatTensor))
                f1score += metrics.f1_score(top_class.cpu(), targets.view(*top_class.shape).long().cpu(), average='weighted')
            
        net.train() # reset to train mode after iterationg through validation data
        print(batch_size)        
        print("Epoch: {}/{}...".format(e+1, epochs),
        "Train Loss: {:.4f}...".format(np.mean(train_losses)),
        "Val Loss: {:.4f}...".format(np.mean(val_losses)),
        "Val Acc: {:.4f}...".format(accuracy/(len(X_test)//batch_size)),
        "F1-Score: {:.4f}...".format(f1score/(len(X_test)//batch_size)))
print(len(X_test))

train(net)

1741
10
Epoch: 1/100... Train Loss: 2.5947... Val Loss: 2.6246... Val Acc: 0.1178... F1-Score: 0.1954...
10
Epoch: 2/100... Train Loss: 2.5760... Val Loss: 2.6020... Val Acc: 0.0914... F1-Score: 0.1424...
10
Epoch: 3/100... Train Loss: 2.3363... Val Loss: 2.1664... Val Acc: 0.3017... F1-Score: 0.3480...
10
Epoch: 4/100... Train Loss: 1.9589... Val Loss: 1.8099... Val Acc: 0.3994... F1-Score: 0.4396...
10
Epoch: 5/100... Train Loss: 1.5708... Val Loss: 1.3915... Val Acc: 0.5103... F1-Score: 0.5380...
10
Epoch: 6/100... Train Loss: 1.2610... Val Loss: 1.1669... Val Acc: 0.5810... F1-Score: 0.5981...
10
Epoch: 7/100... Train Loss: 1.0924... Val Loss: 1.1054... Val Acc: 0.6029... F1-Score: 0.6272...
10
Epoch: 8/100... Train Loss: 0.9773... Val Loss: 1.0205... Val Acc: 0.6178... F1-Score: 0.6393...
10
Epoch: 9/100... Train Loss: 0.8891... Val Loss: 0.9824... Val Acc: 0.6471... F1-Score: 0.6729...
10
Epoch: 10/100... Train Loss: 0.8007... Val Loss: 0.9710... Val Acc: 0.6557... F1-Score: 0.65

KeyboardInterrupt: 